# Lab 9: SVD/PCA Classification for Voice Commands

### EECS16B: Designing Information Devices and Systems II, Summer 2022

Written by Nathaniel Mailoa and Emily Naviasky (2016), Updated by Julian Chan (2018), Peter Schafhalter (2019),  Zhongkai Wang (2021), Bozhi Yin (2021).

nmailoa@berkeley.edu &emsp; enaviasky@berkeley.edu &emsp; julianchan0928@berkeley.edu &emsp; pschafhalter@berkeley.edu &emsp; zhongkai@berkeley.edu &emsp; bozhi_yin@berkeley.edu

Updated by Bozhi Yin, Steven Lu, Megan Zeng, and Ke Wang (2022)

Updated by Mingyang Wang (2022)

## Table of Contents

* [Lab Note](https://drive.google.com/file/d/166GyOZngZ4zb9iN-js3I8RqCFlv4TkFK/view?usp=sharing)
* [Lab 9A: SVD/PCA Part 1](#partA)
    * [Part 0: Introduction](#part0)
    * [Part 1: Data Preprocessing](#part1)
    * [Part 2: PCA via SVD](#part2)
    * [Part 3: Clustering Data Points](#part3)
    * [Part 4: Testing Your Classifier](#part4)
    * [Part 5: Checkoff](#part5)
* [Lab 9B: SVD/PCA Part 2](#partB)
    * [Part 6: Record your own data](#part6)
    * [Part 7: Run Classifier on Your Own Data](#part7)
    * [Part 8: Tuning for Best Performance](#part8)
    * [Part 9: Arduino Implementation of PCA Classify](#part9)
    * [Part 10: Checkoff](#part10)
    
## <span style="color:#ba190f"> You need to run the Python scripts one by one, or errors will show up as they rely on the variables defined sequentially!!

## <span style="color:#ba190f"> DO NOT include units when submitting your answers on Gradescope! ONLY include the numerical value rounded to the number of decimal places specified in each question, in the units specified in the question. DO NOT include letters, words, etc. If a question involves entering an answer that is not a numerical value, the format of the answer will be clearly specified in that question.

<a id='partA'></a>
# Lab 9A: SVD/PCA
----
<a id='part0'></a>
## Part 0: Introduction
----

SIXT33N is an obedient little robot that will follow the directions that you tell it. There are four moves that SIXT33N can make: move straight, move straight slowly, turn right, and turn left. However, SIXT33N does not speak human languages, and some words, like "left" and "right", sound very similar (a strong single syllable), while other words are easy to distinguish. Your job in this phase is to find four command words that are easy for SIXT33N to tell apart (consider syllables and intonation).

In this lab, you will develop the principal component analysis (PCA) classifier that allows SIXT33N to tell the difference between the four commands. You will examine several different words, and determine which ones will be easiest to sort by PCA.

**For more background on the connection between SVD and PCA, please read the [lab note](https://www.eecs16b.org/lab/notes/lab_note9.pdf).**


## Pre-Lab

### Complete the pre-lab assignment before doing the lab. For all students, submit your answers to the Gradescope assignment "Pre-Lab 9: Classification". Pre-Lab 9 is due on Tuesday, July 26 at 11:59pm. No late submissions will be accepted.


### Side Note: Datasets in Machine Learning Applications
It is common practice, especially in machine learning applications, to split a dataset into a training set and a smaller test set (some common ratios for train:test are 80:20 or 70:30) when trying to make data-driven predictions/decisions. In this lab, we will collect data and split our dataset into 70% training data and 30% test data. 

### Overview of Classification Procedure
Once you have some sample data collected, we will:
1. Split our data into 2 sets: train_data and test_data
2. Perform PCA and look at how well it separates the train_data 
3. Once you have a set of four words that you like, you will compute the means for each of those four words in the PCA basis. We will classify each word according to which mean it is closest in Euclidean distance to. 
4. To see how well our classifier does on data it has never seen before (this is called generalization in machine learning), we will project test_data onto the same PCA basis as train_data, and find the mean that is closest in Euclidean distance to each data point. 
5. Check the classifier's accuracy, you will port the classifier into the Arduino in Lab 9B.

<b>
The goals of this phase are as follows:
    
- Generate envelope and utilize threshold to get snippets
- PCA + Classifier (4 commands)
- Check accuracy
    
</b>

### Property of speech signals

When humans distinguish words, they listen for temporal and frequency differences to determine what is being said. However, SIXT33N does not have the memory or the processing power to distinguish words nearly as well as our human brains, so we will have to choose much simpler features for SIXT33N to look at (syllables, intonation, magnitude).

When you think of speech signals, you might notice that the shape of the speech wave is a very distinctive part of each word. Taking just the shape of the magnitude of a signal is called enveloping, exemplified in the image below. We want to do some filtering to retrieve the envelope of the audio signal. We train the PCA off of these envelopes and build a classifier to classify new data points.

<center>
<img width="400px" src="images/proj-envelope.png">
</center>


### Choose commands (words)

Keeping in mind that the words that look most different have different shapes or different amplitudes varied over time (like the picture below). 

- **In this phase, we pre-define four words for you, they are "left", "shoe", "meat" and "potato."** However, in Lab 9B, you are free to choose your own words.

For the Gradescope questions, we provide you the "golden" examples (wav audio files in example_wav folder) to make sure everyone get the same answers. 

- **For this part of the lab, you need use these examples to get familar with the procedure. Then record your own commands and go through it again.**

<center>
<img width="400px" src="images/proj-waveform.png">
</center>



<a id='part1'></a>
## <span style="color:navy">Part 1: Preprocessing Data</span>
----
Before we use the recorded wav file for PCA, we must first process it. The following figure shows the example sound waveform, **where the same word is recorded every 2 seconds for 30 times.** 

<center>
<img width="800px" src="images/recording_waveform.png">
</center>

It is not necessary for you to understand the enveloping function well enough to implement it (since we have already done it for you), but just in case you are curious the enveloping function (`read_wav` function in the following script) is described in the following pseudocode:

<code><b>Enveloping function</b>
Import wav file into numpy data.
For every 2 seconds, divide it to 200 blocks. There are nearly 200*30 blocks in total.
For each chunk:
    Find the mean of the chunk.
    Subtract each sample by the mean.
    Find the sum of the absolute value of each sample.
Split the whole data to 30 parts. Each includes one recording of the word.  
</code>


What you really need to know, however, is what the enveloped signal looks like for each word. Spend a little time looking at the data in the Python plots below.

### 1.1 Load Data from wav files

In the following script, we

1. Read the example *wav* files and get its envolope;
2. Split the data set into training and test set;
3. And read the training set.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import csv
import utils
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

# Recording parameters
counts = 30       # total counts

# Load data from wav
###########################################################
# Update with your own recording wav files for Part 2
###########################################################
word1_raw = utils.read_wav("example_wav/left.wav", counts)
word2_raw = utils.read_wav("example_wav/shoe.wav", counts)
word3_raw = utils.read_wav("example_wav/meat.wav", counts)
word4_raw = utils.read_wav("example_wav/potato.wav", counts)
# word4_raw = utils.read_wav("example_wav/boomboom.wav", counts)

# Split the data into training and test set
train_test_split_ratio = 0.7
word1_raw_train, word1_raw_test = utils.train_test_split(word1_raw, train_test_split_ratio)
word2_raw_train, word2_raw_test = utils.train_test_split(word2_raw, train_test_split_ratio)
word3_raw_train, word3_raw_test = utils.train_test_split(word3_raw, train_test_split_ratio)
word4_raw_train, word4_raw_test = utils.train_test_split(word4_raw, train_test_split_ratio)

# Take the same number of readings for all words to be fair
num_samples_train = min(np.shape(word1_raw_train)[0], np.shape(word2_raw_train)[0], np.shape(word3_raw_train)[0], np.shape(word4_raw_train)[0])
word1_raw_train = word1_raw_train[:num_samples_train,:]
word2_raw_train = word2_raw_train[:num_samples_train,:]
word3_raw_train = word3_raw_train[:num_samples_train,:]
word4_raw_train = word4_raw_train[:num_samples_train,:]

4. Plot the data and get a feel for how it looks enveloped. **<span style="color:red">Important: It's okay if the data isn't aligned. The code in the next part will automatically align the data.</span> Note that the fourth word "potato" is not a great command (why?).** We use it to create some errors intentionally when testing the classifier. 

In [ ]:
# Plot all training samples
plt.plot(word1_raw_train.T)
plt.show()
plt.plot(word2_raw_train.T)
plt.show()
plt.plot(word3_raw_train.T)
plt.show()
plt.plot(word4_raw_train.T)
plt.show()

## Questions
<span style="color:#075a04"> **1. When the split ratio is 0.7 (splitting our dataset into 70% training data and 30% test data), how many word samples do we use for the test set in each word? <span style="color:#ba190f"> Enter integer value.** 

< YOUR ANSWER ON GRADESCOPE > 


<span style="color:#075a04"> **2. In the procedure above, we choose the split ratio being 0.7. If we change the split ratio to 0.6, which of the following statement is correct? Choose all the answers that are correct.** 

- A. The classifier gets more training with the data sets.
- B. The classifier gets less training with the data sets.
- C. The classifier gets more test with the data sets.
- D. The classifier gets less test with the data sets.

< YOUR ANSWER ON GRADESCOPE >
    

<a id='part1.2'></a>
### 1.2 Align Audio Recordings

As you can see above, the speech is only a small part of the 2 second window, and each sample starts at different times. PCA is not good at interpreting delay, so we need to somehow start in the same place each time and capture a smaller segment of the 2 second sample where the speech is present. To do this, we will use a thresholding algorithm.

First, we define a **`threshold`** relative to the maximum value of the data. We say that any signal that crosses the threshold is the start of a speech command. In order to not lose the first couple samples of the speech command, we say that the command starts **`pre_length`** samples before the threshold is crossed. We then take a window of the data that is **`length`** long, and try to capture the entire sound of the command in that window.

<b>Play around with the parameters `length`, `pre_length` and `threshold`</b> in the cells below to find appropriate values corresponding to the voice and chosen commands. You should see the results and how much of your command you captured in the plots generated below. When you are satisfied, note down the values of `length`, `pre_length` and `threshold`.

**For the "golden" examples, let's use `length` = 40, `pre_length` = 5, and `threshold` = 0.5 to get the Gradescope answers.** You are free to change these parameters when running the code on your own data.

In [ ]:
def get_snippets(data, length, pre_length, threshold):
    """Attempts to align audio samples in data.
    
    Args:
        data (np.ndarray): Matrix where each row corresponds to a recording's audio samples.
        length (int): The length of each aligned audio snippet.
        pre_length (int): The number of samples to include before the threshold is first crossed.
        threshold (float): Used to find the start of the speech command. The speech command begins where the
            magnitude of the audio sample is greater than (threshold * max(samples)).
    
    Returns:
        Matrix of aligned recordings.
    """
    assert isinstance(data, np.ndarray) and len(data.shape) == 2, "'data' must be a 2D matrix"
    assert isinstance(length, int) and length > 0, "'length' of snippet must be an integer greater than 0"
    assert 0 <= threshold <= 1, "'threshold' must be between 0 and 1"
    snippets = []

    # Iterate over the rows in data
    for recording in data:
        # Find the threshold
        recording_threshold = threshold * np.max(recording)

        # Figure out when interesting snippet starts
        i = pre_length
        while recording[i] < recording_threshold:
            i += 1
            
        snippet_start = min(i - pre_length, len(recording) - length)
        snippet = recording[snippet_start:snippet_start + length]

        # Normalization
        snippet = snippet / np.sum(snippet) * 10000
        snippet = snippet.astype(int)
        
        snippets.append(snippet)

    return np.vstack(snippets)

In [ ]:
length = 40 # Default: 40        # Keep for example wav files, adjust this with your recordings
pre_length = 5 # Default: 5      # Keep for example wav files, adjust this with your recordings
threshold = 0.5 # Default: 0.5   # Keep for example wav files, adjust this with your recordings

word1_processed_train = get_snippets(word1_raw_train, length, pre_length, threshold)
plt.plot(word1_processed_train.T)
plt.show()
plt.figure()
word2_processed_train = get_snippets(word2_raw_train, length, pre_length, threshold)
plt.plot(word2_processed_train.T)
plt.show()
word3_processed_train = get_snippets(word3_raw_train, length, pre_length, threshold)
plt.plot(word3_processed_train.T)
plt.show()
plt.figure()
word4_processed_train = get_snippets(word4_raw_train, length, pre_length, threshold)
plt.plot(word4_processed_train.T)
plt.show()

You should now see a mostly organized set of samples for each word. Can you tell the which word is which just by the envelope? Can you tell them apart? If you can't tell the words apart, then PCA will have a difficult time as well.

## Questions

<span style="color:#075a04"> **3. For the audio data alignment script, if we only make the `threshold` smaller, for example, from 0.5 to 0.1. Which of the following statements is correct? Choose all the answers that are correct.** 

- A. The snippets of the audio data start later in the two second time window.
- B. The snippets of the audio data start earlier in the two second time window.
- C. The snippets data is more sensitive to noise and may get aligned at wrong positions.
- D. The snippets data is less sensitive to noise and are easier to get aligned.
    
< YOUR ANSWER ON GRADESCOPE > 
    
    
<span style="color:#075a04"> **4. For the example data sets, if we change the `pre_length` larger, for example, from 5 to 20. Which of the following statements is correct? Choose all the answers that are correct.** 

- A. The beginning of snippets contains more zeros.
- B. The beginning of snippets contains less zeros.
- C. The relative positions of the snippets to each other are changed.
- D. The relative positions of the snippets to each other do not change.
    
< YOUR ANSWER ON GRADESCOPE > 
    
    
<span style="color:#075a04"> **5. From the example data set we provide, which word is not aligned as well as the others?**
    
- A. Word1 - "left"
- B. Word2 - "shoe"
- C. Word3 - "meat"
- D. Word4 - "potato"
   
< YOUR ANSWER ON GRADESCOPE > 
    

<a id='part2'></a>
## <span style="color:navy">Part 2: PCA via SVD</span>
----

### 2.1 Generate and Preprocess PCA Matrix

Now that we have our data in a nice format, we can build the PCA input matrix from that data by **stacking all the data vertically**. The function <b>`np.vstack`</b> might be helpful here.

In [ ]:
# YOUR CODE HERE
processed_A = ...

print(processed_A.shape)

The first step of PCA is zero-mean your data as `demeaned_A`. Centering the data can be helpful to obtain principial components that are representative of the shape of the variations in the data. Please note that you want to **get the mean of the features (or mean of each timestep).** The function `np.mean` might be helpful here.

In [ ]:
# Zero-mean the matrix A
# YOUR CODE HERE
mean_vec = ...        # Hint: what should the axis be if we want to get the mean value of each timestep?
demeaned_A = ...

print(mean_vec.shape)
print(processed_A.shape)

### 2.2 Principal Component Analysis

Firstly, write code below to perform SVD on the demeaned matrix A. The function `np.linalg.svd` might be helpful at here.

In [ ]:
# Take the SVD of matrix demeaned_A
# YOUR CODE HERE #
U, S, Vt = ...       # Line 1 for parameter tuning

print(S.shape)
print(U.shape)
print(Vt.shape)

Plot the **sigma values**, and project on to the principal components (Hint: Use `plt.stem` for a stem plot, and use `plt.xlim` to define x axis limits). 

In [ ]:
# Plot out the sigma values (Hint: Use plt.stem for a stem plot)
# YOUR CODE HERE #
...
plt.xlim([-1,10])
plt.show()

Take a look at your sigma values. They should show you very clearly how many principal components you need.

**<span style="color:red">How many principal components do you need? Given that you are sorting 4 words, what is the the number you expect to need?</span>** 

There is no correct answer here. We can pick as many principal components onto which we project our data to get the "best" separation (most variance), but at some point, the cost-benefit isn't worth selecting an extra basis vector. For example, in our project, we are loading these basis vectors onto the Arduino, and we can only store 2-3 principal components before we run into memory issues.

### 2.3 Choosing a Basis using Principal Components

In the example procedure, we are going to choose `two` principal components in $V_t$ as `new_basis` for our example data sets. You can also choose `three` principal components later and compare the classification results. **Please note that how $V_t$ is organized (Hint: matrix transpose may be needed).**

In [ ]:
# Choose and plot the principal component(s) (in Vt)
num_pca = 2             # Numer of PCA vectors. Leave it as 2 for given data in Lab 9A; You can use 2 or 3 for your own data in Lab 9B.
new_basis = Vt[:num_pca].T  # Line 2 for parameter tuning. This should be the basis containing your principal components. 

plt.plot(new_basis)
print(new_basis.shape)

In [ ]:
def plot_3D(ax,data, view_from_top=False, m = 'o', si = 20):
    colors = ['blue', 'red', 'green', 'orange']
    for dat, color in zip(data, colors):
        Axes3D.scatter(ax, *dat.T, c=color, marker = m, s=si)
    if view_from_top:
        ax.view_init(elev=90.,azim=0)  # Move perspective to view from top

Now you need to fill the code to project the data in the matrix A (`demeaned_A`) onto the new basis and plot it (Hint: the `np.dot` function might be helpful). Do you see clustering? Do you think you can separate the data easily? If not, you might need to try new words.

In [ ]:
# Project the data onto the new basis
# YOUR CODE HERE
proj = ...              # Line 3 for parameter tuning
print(proj.shape)

if num_pca == 3: 
    fig=plt.figure(figsize=(10,7))
    ax = fig.add_subplot(111, projection='3d')
    plot_3D(ax,[proj[:num_samples_train], proj[num_samples_train:2*num_samples_train], proj[2*num_samples_train:3*num_samples_train], proj[3*num_samples_train:4*num_samples_train]])
    plt.show()
elif num_pca == 2:  
    plt.figure()
    plt.scatter(proj[0:num_samples_train,0], proj[0:num_samples_train,1], c=['blue'], edgecolor='none')
    plt.scatter(proj[num_samples_train:num_samples_train*2,0], proj[num_samples_train:num_samples_train*2,1], c=['red'], edgecolor='none')
    plt.scatter(proj[num_samples_train*2:num_samples_train*3,0], proj[num_samples_train*2:num_samples_train*3,1], c=['green'], edgecolor='none')
    plt.scatter(proj[num_samples_train*3:num_samples_train*4,0], proj[num_samples_train*3:num_samples_train*4,1], c=['orange'], edgecolor='none')
    plt.legend(['word1', 'word2', 'word3', 'word4'],loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()
else:
    print("Error: Please only use 2 or 3 PCA vectors")

Your data might look noisy, and might not classify perfectly. That is completely okay, we are just looking for good enough. Like many AI applications, this is noisy data that we are classifying so some error in classification is okay. The important part is if you think that you can see some clustering. 

Once you think you have decent clustering, you can move on to getting your code to automate classification and you will make up for some of the error there, too.

## Questions
    
<span style="color:#075a04"> **6. With the example data sets, what is the first value of the sigma vector `S`? <span style="color:#ba190f"> Enter numerical value with two decimal places (e.g. 3.14).**

< YOUR ANSWER ON GRADESCOPE > 
    
    
<span style="color:#075a04"> **7. With the example data sets, what is the ratio between the second and third values (second value/third value) of the sigma vector `S`? <span style="color:#ba190f"> Enter numerical value with two decimal places (e.g. 3.14).** 

< YOUR ANSWER ON GRADESCOPE > 
     
    
<span style="color:#075a04"> **8. For the example data sets, from the scatter plot, which of the following two words are most near to each other and maybe hard to distinguish?** 

- A. Word 1 
- B. Word 2 
- C. Word 3 
- D. Word 4 
 
< YOUR ANSWER ON GRADESCOPE > 


<a id='part3'></a>
## <span style="color:navy">Part 3: Clustering Data Points</span>
----

#### Implement `find_centroids`  below which finds the center of each cluster.

**This function finds the center of each cluster (or each word) by taking the mean of all points in a cluster (Hint: what's the size of our training set?).** We will use the centroids as reference to find the Euclidean distance of new test datapoints to each centroid, and determine which cluster or word each test datapoint belongs to.

In [ ]:
def find_centroids(clustered_data):
    """Find the center of each cluster by taking the mean of all points in a cluster.
    It may be helpful to recall how you constructed the data matrix (e.g. which rows correspond to which word)
    
    Parameters:
        clustered_data: the data already projected onto the new basis
        
    Returns: 
        The centroids of the clusters
    """
    # YOUR CODE HERE
    # The variable `num_samples_train` may be useful here
    centroids = ...
    ...
    
    return centroids

In [ ]:
# Determine the centroids of each cluster
centroids = find_centroids(proj)         
print(centroids)

In [ ]:
centroid1 = centroids[0]
centroid2 = centroids[1]
centroid3 = centroids[2]
centroid4 = centroids[3]
centroid_list = np.vstack([centroid1, centroid2, centroid3, centroid4])

print('The first centroid is at: ' + str(centroid1))
print('The second centroid is at: ' + str(centroid2))
print('The third centroid is at: ' + str(centroid3))
print('The fourth centroid is at: ' + str(centroid4))

if num_pca == 3:
    fig=plt.figure(figsize=(10,7))
    ax = fig.add_subplot(111, projection='3d')
    plot_3D(ax,[proj[:num_samples_train], proj[num_samples_train:2*num_samples_train], proj[2*num_samples_train:3*num_samples_train], proj[3*num_samples_train:4*num_samples_train]], view_from_top=False)
    plot_3D(ax,[np.array([centroids[0]]), np.array([centroids[1]]), np.array([centroids[2]]), np.array([centroids[3]])], False, '*', 300)
    plt.title("Training Data")
    fig.show()
elif num_pca == 2:  
    plt.figure()
    plt.scatter(proj[0:num_samples_train,0], proj[0:num_samples_train,1], c=['blue'], edgecolor='none')
    plt.scatter(proj[num_samples_train:num_samples_train*2,0], proj[num_samples_train:num_samples_train*2,1], c=['red'], edgecolor='none')
    plt.scatter(proj[num_samples_train*2:num_samples_train*3,0], proj[num_samples_train*2:num_samples_train*3,1], c=['green'], edgecolor='none')
    plt.scatter(proj[num_samples_train*3:num_samples_train*4,0], proj[num_samples_train*3:num_samples_train*4,1], c=['orange'], edgecolor='none')

    plt.scatter(centroid_list[:,0], centroid_list[:,1], c=['blue', 'red', 'green', 'orange'], marker='*', s=300)
    plt.legend(['word1', 'word2', 'word3', 'word4'],loc='center left', bbox_to_anchor=(1, 0.5))
    plt.title("Training Data")
    plt.show()
else:
    print("Error: Please only use 2 or 3 PCA vectors")

## Questions

<span style="color:#075a04"> **9. With the example data sets, what is the first coordinate of centroid coordinate of word4? <span style="color:#ba190f">Enter numerical value with two decimal places (e.g. 3.14).** 

< YOUR ANSWER ON GRADESCOPE > 
    
    
<span style="color:#075a04"> **10. With the example data sets, what is Euclidean distance between the centroids of word1 and word2? <span style="color:#ba190f">Enter numerical value with two decimal places (e.g. 3.14).** 

< YOUR ANSWER ON GRADESCOPE > 
    
    
<span style="color:#075a04"> **11. With the example data sets, which of the following two words have the smallest Euclidean distance between the centroids?**
    
- A. Word 1 
- B. Word 2 
- C. Word 3 
- D. Word 4
    
< YOUR ANSWER ON GRADESCOPE > 
    

<a id='part4'></a>
## <span style="color:navy">Part 4: Testing Your Classifier</span>
----

## 4.1 Preprocess Test Data 

Great! We now have the means (centroid) for each word. Now let's see how well our test data performs. Recall that we will classify each data point according to the centroid that it is closest in Euclidean distance to. 

Before we perform classification, we need to do the same preprocessing to the test data that we did to the training data (enveloping, demeaning, projecting onto the PCA basis). You have already written most of the code for this part. However, note the difference in variable names as we are now working with test data.

First let's look at what our raw test data looks like.

In [ ]:
# Take the same number of readings for all words to be fair
num_samples_test = min(np.shape(word1_raw_test)[0], np.shape(word2_raw_test)[0], np.shape(word3_raw_test)[0], np.shape(word4_raw_test)[0])
word1_raw_test = word1_raw_test[:num_samples_test,:]
word2_raw_test = word2_raw_test[:num_samples_test,:]
word3_raw_test = word3_raw_test[:num_samples_test,:]
word4_raw_test = word4_raw_test[:num_samples_test,:]

In [ ]:
# Plot all test samples
plt.plot(word1_raw_test.T)
plt.show()
plt.plot(word2_raw_test.T)
plt.show()
plt.plot(word3_raw_test.T)
plt.show()
plt.plot(word4_raw_test.T)
plt.show()

Then, let's perform alignment and trimming of our test data.

In [ ]:
word1_processed_test = get_snippets(word1_raw_test, length, pre_length, threshold)
plt.plot(word1_processed_test.T)
plt.show()
plt.figure()
word2_processed_test = get_snippets(word2_raw_test, length, pre_length, threshold)
plt.plot(word2_processed_test.T)
plt.show()
word3_processed_test = get_snippets(word3_raw_test, length, pre_length, threshold)
plt.plot(word3_processed_test.T)
plt.show()
plt.figure()
word4_processed_test = get_snippets(word4_raw_test, length, pre_length, threshold)
plt.plot(word4_processed_test.T)
plt.show()

Similar as training data sets, construct the PCA matrix by stacking all the test data vertically. The function `np.vstack` might be helpful here.

In [ ]:
# YOUR CODE HERE
processed_A_test = ...

## 4.2 Project Test Data on PCA Basis

Now we will do something slightly different. Previously, you projected data using some projection matrix with $ P(x - \bar{x}) $, where $\bar{x}$ is the mean vector.

We can rewrite this operation as $ P(x - \bar{x}) = Px - P \bar{x} = Px - \bar{x}_{\text{proj}} $ where $ \bar{x}_{\text{proj}} = P \bar{x} $.

Why might we want to do this? We'll later perform these operations on Arduino, which has limited memory, so we want to store as little data as possible. Instead of storing a length $n$ vector $\bar{x}$, we can precompute $ \bar{x}_{\text{proj}} $ (length 2 or 3) and store that instead!

Compute $ \bar{x}_{\text{proj}} $ using the **mean vector** (from part 2.1) as the one computed with the training data. Hint: use the `np.dot` function to project the **mean vector** onto the PCA basis.

In [ ]:
# YOUR CODE HERE
projected_mean_vec = ...     # Line 4 for parameter tuning

print(projected_mean_vec)

Project the test data onto the **same PCA basis** as the one computed with the training data. Hint: use the `np.dot` function.

In [ ]:
# YOUR CODE HERE
projected_A_test = ...       # Line 5 for parameter tuning

Zero-mean the projected test data using the **`projected_mean_vec`**.

In [ ]:
# YOUR CODE HERE
proj = ...                   # Line 6 for parameter tuning

Plot the projections to see how well your test data clusters in this new basis. This will give you an idea of how well your test data will classify.

In [ ]:
if num_pca == 3: 
    fig=plt.figure(figsize=(10,7))
    ax = fig.add_subplot(111, projection='3d')
    plot_3D(ax,[proj[:num_samples_test], proj[num_samples_test:2*num_samples_test], proj[2*num_samples_test:3*num_samples_test], proj[3*num_samples_test:4*num_samples_test]], view_from_top=False)
    plot_3D(ax,[np.array([centroid_list[0]]), np.array([centroid_list[1]]), np.array([centroid_list[2]]), np.array([centroid_list[3]])], False, '*', 300)
    plt.title("Test Data")
    fig.show()
elif num_pca == 2:  
    plt.figure()
    plt.scatter(proj[0:num_samples_test,0], proj[0:num_samples_test,1], c=['blue'], edgecolor='none')
    plt.scatter(proj[num_samples_test:num_samples_test*2,0], proj[num_samples_test:num_samples_test*2,1], c=['red'], edgecolor='none')
    plt.scatter(proj[num_samples_test*2:num_samples_test*3,0], proj[num_samples_test*2:num_samples_test*3,1], c=['green'], edgecolor='none')
    plt.scatter(proj[num_samples_test*3:num_samples_test*4,0], proj[num_samples_test*3:num_samples_test*4,1], c=['orange'], edgecolor='none')
    plt.legend(['word1', 'word2', 'word3', 'word4'],loc='center left', bbox_to_anchor=(1, 0.5))

    plt.scatter(centroid_list[:,0], centroid_list[:,1], c=['blue', 'red', 'green', 'orange'], marker='*', s=300)
    plt.legend(['word1', 'word2', 'word3', 'word4'],loc='center left', bbox_to_anchor=(1, 0.5))
    plt.title("Test Data")
    plt.show()
else:
    print("Error: Please only use 2 or 3 PCA vectors")

Now that we have some idea of how our test data looks on this new basis, let's see how our data actually performs. **Implement the classify function that takes in a data point (AFTER enveloping is applied) and returns which word number it belongs to depending on which centroid the data point is closest in Euclidean distance to.** You may find the `np.argmin`, `np.linalg.norm`, and the `map` functions useful.

In [ ]:
def classify(data_point, new_basis, projected_mean_vec, centroid_list):
    """Classifies a new voice recording into a word.
    
    Args:
        data_point: new data point vector before demeaning and projection
    Returns:
        Word number (should be in {1, 2, 3, 4})
    Hint:
        Remember to use 'projected_mean_vec'!
    """
    # YOUR CODE HERE
    projected_data_point = ...
    demeaned = ...
    # TODO: classify the demeaned data point by comparing against the centroids
    # You can use a loop or map function to get the distances
    ...
    return ...

In [ ]:
# Try out the classification function
print(classify(processed_A_test[0,:], new_basis, projected_mean_vec, centroid_list)) # Modify to use other vectors

**Our goal is 80% accuracy for each word.** The following code applies the `classify` function to each sample and compute the accuracy for each word.

In [ ]:
# Try to classify the whole A matrix
correct_counts = np.zeros(4)

for (row_num, data) in enumerate(processed_A_test):
    word_num = row_num // num_samples_test + 1
    if classify(data, new_basis, projected_mean_vec, centroid_list) == word_num:
        correct_counts[word_num - 1] += 1
    else:
        print("For word {}, we got word {}.".format(word_num, classify(data, new_basis, projected_mean_vec, centroid_list)))
        
for i in range(len(correct_counts)):
    print("Percent correct of word {} = {}%".format(i + 1, 100 * correct_counts[i] / num_samples_test))

## Questions

<span style="color:#075a04"> **12. With the example data, what is the first coordinate of `projected_mean_vec`? <span style="color:#ba190f">Enter a numerical value with two decimal places (e.g. 3.14).**

< YOUR ANSWER ON GRADESCOPE > 
    
    
<span style="color:#075a04"> **13. With the example data sets, what is the accuracy of word1 in percentage? <span style="color:#ba190f">Enter an integer value.**
 
< YOUR ANSWER ON GRADESCOPE >   
    
    
<span style="color:#075a04"> **14. With the example data sets, what is the accuracy of word4 in percentage? <span style="color:#ba190f">Enter an integer value.**
  
< YOUR ANSWER ON GRADESCOPE > 
  
    
<span style="color:#075a04"> **15. If the classifier get errors for word4 in previous question, which of the following is the classifier's prediction? If you do not get a error, choose `None`.**
    
- A. Word 1 
- B. Word 2 
- C. Word 3 
- D. None
    
< YOUR ANSWER ON GRADESCOPE > 
       

<a id='part5'></a>
## <span style="color:#ba190f">Part 5: CHECKOFF </span> 
-----

### Submit your answers to Questions 1-15 to the Gradescope assignment "Lab 9: Classification." Lab 9 is due on Sunday, April 24 at 11:59pm. No late submissions will be accepted.

<a id='partB'></a>
# Lab 9B: SVD/PCA Part2
----
<a id='part6'></a>
## <span style="color:navy">Part 6: Record your own data</span>

Now, you're going to **record 40 audio samples for each of your 4 chosen words.** If you are working with a partner, we recommend each partner records 2 words to maximize variance between the recorded signals. Again, think about how to make the words distinct from others (syllable count, intontation). The reason for using 40 samples, instead of 30 is that it allows you to have 2 misclassifications under the target classification accuracy of 80%.

"Good" Audio data has a high signal-to-noise ratio. Recording words while far away from the microphone may cause your intended word to blend in with background noise. However "oversaturation" of the audio signal (speaking too loudly and/or too closely into the mic) will distort the signal (Why?). It's recommended to record your own words with a headphone microphone to get good audio data.

To record your own `wav` files, we suggest you use the [online recorder](https://online-voice-recorder.com). 

**For each chosen word, do the following:**
1. Click on record button on the webpage, and come back to this page. 
2. **Run the script in next cell, which will print out numbers from 1 to 40 in intervals of 2 seconds. When you see the numbers appear, say the word you want to record.**
    - Since the recording window is 2 seconds, you want to finish the word in this period. 
    - **Pronounce the word consistently and try to speak around the same time relative to when the number appears.** This will help you collect data that is less "noisy" which will result in better classification.
    
    
3. Once you've recorded 40 times of the word, the script will stop automatically. **Then go back to the webpage and stop the recording.** You should get webpage like the following.

<center>
<img width="800px" src="images/recording.png">
</center>

4. Change the start time near your first sample and the end time near your last sample. **You want to remove the noise signals before the first sample and after the last sample. And also, it's recommended to leave some amount of empty regions at the beginning and end.** The total recording time should be around 80 seconds.

5. When you save the file, it is the `mp3` type. Use the [audio converter](https://online-audio-converter.com) to convert it to `wav` file with the options shown as below.

<center>
<img width="600px" src="images/convert_to_wav.png">
</center>

6. Save the converted `wav` files into folder `recording_wav`.

If you cannot get access to the website, you can record with your phone. But you need to convert your recording to a `wav` format, and clip it to remove the noise signals before the first sample and after the last sample. Another website to convert the audio files are [Convert-to-WAV](https://audio.online-convert.com/convert-to-wav) and the website to trim the `wav` file is [mp3cutter](https://www.mp3cutter.com). You can also find other useful websites to do this.


In [ ]:
# YOUR CODE HERE: Fill in the four words you recorded
all_words_arr = ["left", "shoe", "meat", "potato"]

In [ ]:
counts = 40       
record_time = 2   # record time for each sample of the word

utils.recording_timer(counts, record_time)

<a id='part7'></a>
## <span style="color:navy">Part 7: Run Classifier on Your Own Data</span>
----
1. Run codes in the following cell to load data from your recorded wav files
1. Re-run section [Part 1.2](#part1.2) to align the samples. Tweak `length`, `pre_length` and `threshold` to make sure they are aligned and distinguishable in time waveforms.
2. Re-run the whole SVD/PCA procedure (Part 2-4) again to classify your own data.

In [ ]:
###########################################################
# Load data from your recorded wav files
###########################################################
word1_raw = utils.read_wav("recording_wav/%s.wav" % all_words_arr[0], counts)
word2_raw = utils.read_wav("recording_wav/%s.wav" % all_words_arr[1], counts)
word3_raw = utils.read_wav("recording_wav/%s.wav" % all_words_arr[2], counts)
word4_raw = utils.read_wav("recording_wav/%s.wav" % all_words_arr[3], counts)

# Split the data into training and test set
train_test_split_ratio = 0.7
word1_raw_train, word1_raw_test = utils.train_test_split_random(word1_raw, train_test_split_ratio)
word2_raw_train, word2_raw_test = utils.train_test_split_random(word2_raw, train_test_split_ratio)
word3_raw_train, word3_raw_test = utils.train_test_split_random(word3_raw, train_test_split_ratio)
word4_raw_train, word4_raw_test = utils.train_test_split_random(word4_raw, train_test_split_ratio)

# Take the same number of readings for all words to be fair
num_samples_train = min(np.shape(word1_raw_train)[0], np.shape(word2_raw_train)[0], np.shape(word3_raw_train)[0], np.shape(word4_raw_train)[0])
word1_raw_train = word1_raw_train[:num_samples_train,:]
word2_raw_train = word2_raw_train[:num_samples_train,:]
word3_raw_train = word3_raw_train[:num_samples_train,:]
word4_raw_train = word4_raw_train[:num_samples_train,:]

## Questions

<span style="color:#075a04"> **1. With your own data set, what is the worst accuracy among four words in percentage? <span style="color:#ba190f">Enter an integer value (No correct answer, just write down what you got).**

< YOUR ANSWER ON GRADESCOPE > 
    
    
<span style="color:#075a04"> **2. Run the script in next cell to convert processed data to csv files and upload them to Gradescope.**
- If you don't feel comfortable uploading your own data, you can pronounce words with an online dictionary (www.dictionary.com) on your cellphone and record the audio based on instructions in Part 6. Don't forget to run Part 7 before converting the CSV file with the following script.
 
< UPLOAD CSV FILES ON GRADESCOPE >      
       

In [ ]:
# Dump preprocessed data to csv file
np.savetxt("./pca_data/%s.csv" % all_words_arr[0], word1_raw, delimiter=",")
np.savetxt("./pca_data/%s.csv" % all_words_arr[1], word2_raw, delimiter=",")
np.savetxt("./pca_data/%s.csv" % all_words_arr[2], word3_raw, delimiter=",")
np.savetxt("./pca_data/%s.csv" % all_words_arr[3], word4_raw, delimiter=",")

<a id='part8'></a>
## <span style="color:navy">Part 8: Tuning for Best Performance</span>
----
In this part, we will optimize our preprocessing parameters (length, pre_length, and threshold) to achieve the best classification rate. **You do not need to understand any of this code for the optimizer. You don't need to re-run any previous parts either, because the optimizer has already done it for you.**

In [ ]:
!pip install optuna
import warnings
import itertools
warnings.filterwarnings('ignore')

Here we define a function the optimizer will use. Copy the lines marked with a comment **"Line # for parameter tuning"** from **the earlier parts of the lab** into the corresponding lines below.

In [ ]:
def load_data(length, pre_length, threshold):
    word1_processed_train = get_snippets(word1_raw_train, length, pre_length, threshold)
    word2_processed_train = get_snippets(word2_raw_train, length, pre_length, threshold)
    word3_processed_train = get_snippets(word3_raw_train, length, pre_length, threshold)
    word4_processed_train = get_snippets(word4_raw_train, length, pre_length, threshold)
    processed_A = np.vstack([word1_processed_train, word2_processed_train, word3_processed_train, word4_processed_train])
    mean_vec = np.mean(processed_A, axis=0)
    demeaned_A = processed_A - mean_vec
    
    U, S, Vt = ...            # Copy Line 1 Here
    new_basis = ...           # Copy Line 2 Here
    
    proj = ...                # Copy Line 3 Here

    centroids = find_centroids(proj) 
    
    word1_processed_test = get_snippets(word1_raw_test, length, pre_length, threshold)
    word2_processed_test = get_snippets(word2_raw_test, length, pre_length, threshold)
    word3_processed_test = get_snippets(word3_raw_test, length, pre_length, threshold)
    word4_processed_test = get_snippets(word4_raw_test, length, pre_length, threshold)
    processed_A_test = np.vstack([word1_processed_test, word2_processed_test, word3_processed_test, word4_processed_test])
    
    projected_mean_vec = ...  # Copy Line 4 Here
    projected_A_test = ...    # Copy Line 5 Here
    proj = ...                # Copy Line 6 Here
    num_samples_test = min(list([word1_raw_test.shape[:][0], word2_raw_test.shape[:][0], word3_raw_test.shape[:][0], word4_raw_test.shape[:][0]]))
    return processed_A_test, new_basis, projected_mean_vec, centroids, num_samples_test

Here we define the objective the optimizer tries to optimize.

In [ ]:
def objective(trial):
    """
    Defines an objective that we try to optimize. In this lab, it's the accuracy for classification. 
    Simply run this block.
    
    Args:
        trial (object): a specific trial with given combinations of parameters
        
    Returns:
        -1 if the given combination of param is not compatible with the model;
        The number of past test cases if fewer than 4 cases are passed. Thus, returns value in [0, 3];
        The average percent correct of word if all 4 test cases are passed. The % sign is omitted.
    """
    # max_length is related to the hardware limitation of Arduino, which should not be changed.
    if num_pca == 3:
        max_length = 50
    else:
        max_length = 65    
        
    length = trial.suggest_int('length', 30, max_length) # Default: 40       # Suggest range for length
    pre_length = trial.suggest_int('pre_length', 0, 9) # Default: 5          # Suggest range for pre_length
    threshold = trial.suggest_float('threshold', 0., 0.99) # Default: 0.5    # Suggest range for threshold
    try:
        data = load_data(length, pre_length, threshold)
        processed_A_test, new_basis, projected_mean_vec, centroids, num_samples_test = data
    except:
        return -1 # the given combination of param is not compatible with the model
    
    correct_counts = np.zeros(4)
    for (row_num, data) in enumerate(processed_A_test):
        word_num = row_num // num_samples_test + 1
        if classify(data, new_basis, projected_mean_vec, centroid_list) == word_num:
            correct_counts[word_num - 1] += 1
                
    num_pass = 0
    total_auc = 0
    for i in range(len(correct_counts)):
        auc = 100 * correct_counts[i] / num_samples_test
        total_auc += auc
        if auc >= 80:
            num_pass += 1
    if num_pass < 4:
        return num_pass
    return total_auc / len(correct_counts)

Now, let's run the optimizer. You do not need to change any of the parameters.

In [ ]:
study = utils.create_study()
study.optimize(objective, n_trials=60, timeout=60) # Number of trials and timeout (in seconds)

In [ ]:
# Print the best stats
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Now that we have our optimized preprocessing parameters, let's try our classification again and see if it improved. Feel free to replace length, pre-length, and threshold with your own values if you find that they work better.

In [ ]:
length = trial.params['length']
pre_length = trial.params['pre_length']
threshold = trial.params['threshold']
processed_A_test, new_basis, projected_mean_vec, centroids, num_samples_test = load_data(length, pre_length, threshold)

correct_counts = np.zeros(4)
correct_counts_tot = 0

for (row_num, data) in enumerate(processed_A_test):
    word_num = row_num // num_samples_test + 1
    if classify(data, new_basis, projected_mean_vec, centroids) == word_num:
        correct_counts[word_num - 1] += 1
        
for i in range(len(correct_counts)):
    print("Percent correct of word {} = {}%".format(i + 1, 100 * correct_counts[i] / num_samples_test))
    correct_counts_tot += correct_counts[i]

You should now be able to meet the **80%** classification accuracy we require (>80% accuracy for each word)! If not, you may need to re-record some of your words or adjust some parameters for the optimizer. Make sure to note down the values of `length`, `pre_length` and `threshold` - <b>you will need to add them to the Arduino sketch later!</b>

## Questions

<span style="color:#075a04"> **3. Check the classification accuracy when using 2 PCA components. Hint: Change the number of PCA components by setting variable `num_pca` in Part 2.3, and then re-run Part 8 to get new results. <span style="color:#ba190f">**

<span style="color:#075a04"> **3.1. What is the worst accuracy among four words in percentage? <span style="color:#ba190f">** Enter a numerical value with two decimal places (e.g. 3.14) (No correct answer, just write down what you got).
    
< YOUR ANSWER ON GRADESCOPE >    

<span style="color:#075a04"> **3.2. Upload a screenshot including the optimum parameter values and classification results of q3 to Gradescope. <span style="color:#ba190f">**
    
< YOUR ANSWER ON GRADESCOPE >

    
<span style="color:#075a04"> **4. Check the classification accuracy when using 3 PCA components. Hint: Change the number of PCA components by setting variable `num_pca` in Part 2.3, and then re-run Part 8 to get new results. <span style="color:#ba190f">**

<span style="color:#075a04"> **4.1. What is the worst accuracy among four words in percentage? <span style="color:#ba190f">** Enter a numerical value with two decimal places(e.g. 3.14) (No correct answer, just write down what you got).
    
< YOUR ANSWER ON GRADESCOPE >    

<span style="color:#075a04"> **4.2. Upload a screenshot including the optimum parameter values and classification results of q4 to Gradescope. <span style="color:#ba190f">**
    
< YOUR ANSWER ON GRADESCOPE >


<a id='part9'></a>
## <span style="color:#ba190f">Part 9: Arduino Implementation of PCA Classifier </span> 
-----
    
### Material

- Recorded words
- TinkerCAD model (from your previous lab)

Your last step will be to implement your <b>data processing</b> and <b>classification</b> (just the projection, not the PCA) in the Arduino sketch <b>`classify.ino`</b>. Since the Arduino environment does not have as many built-in functions as Python, you might have to write out the functions yourself. For example, a dot product should be written as:

`float result = 0;`<br/>
`for (i=0; i<LENGTH; i++){`<br/>
&emsp; `result += vector1[i]*vector2[i];`<br/>
`}`

For debugging purposes, printing to the serial plotter looks like the line below.

<code>Serial.println("I'm being printed!");</code>

There are 3 code blocks (`PCA1/2/3`) that you need to modify.

**`CODE BLOCK PCA1`**
- Copy `SNIPPET_SIZE`, `PRELENGTH` and `THRESHOLD` from **the optimum parameter values found in part 8**.
- Read the following to set `EUCLIDEAN_THRESHOLD`.

**`EUCLIDEAN_THRESHOLD` filters the classification depending on a sample's distance from the closest centroid.** If the L2 norm (distance) is larger than the threshold, your classification algorithm should simply ignore it and wait for the next sample. Look at the plot of sample data and the centroids from the PCA Notebook and approximate a radius around the centroids that capture most of the data. **Try to be conservative - it's better to not classify than to misclassify.**

**`CODE BLOCK PCA2`**
- Copy the PCA vectors, mean, and centroids **that are printed after running the python script below**.
- If you need more than 2 principal components, add a new `pca_vec3` array. 
- Using more principal components increases the dimensionality of the centroids and projections.

**`CODE BLOCK PCA3`**
- This is the actual classification algorithm.
- Project `result`, which will be your test data, onto your new PCA basis.
    - **Since the principal component vectors are orthonormal, they have unit norm and when you perform the projection of `result` onto the principal components, you only need to do the dot products between those vectors using the for loop.**
    - Use the variables `proj1` and `proj2` to store the projection results.
    - Remember to subtract the projected mean vector.
- Classify the projections using the centroids.
    - Find the distance between the projected data point and each centroid using the function `l2_norm` (for 2 principal components) or `l2_norm3` (for 3 principal components). Look up the function definition in the sketch.
    - Out of the 4 centroids, find the one with the smallest L2 norm.
    - Verify this distance is less than `EUCLIDEAN_THRESHOLD`. Hint: Print to the Serial Monitor whenever the best_dist is greater than the Euclidean Threshold, which can give you guidance on how to change the threshold value.
- Print the classification to the serial monitor.

Lastly, underneath `CODE BLOCK PCA2`, replace `result` with one of the test data points printed by the python script below. Now upload the sketch, open the Serial Monitor, and run. And observe how the Arduino classifies your word.

**<span style="color:red">If the Arduino does not classify as well as you think it should, remember to play with the `EUCLIDEAN_THRESHOLD` variables.</span>** To debug the sketch, you can also print out any of the variables you have used. 

Voila! Your SIXT33N can recognize your words!

## Questions

<span style="color:#075a04"> **5. Which of following could be true if we increase the EUCLIDEAN_THRESHOLD? <span style="color:#ba190f">**

 - A. More samples might be misclassified
 - B. Fewer samples might be misclassified
 - C. No impact at all
    
< YOUR ANSWER ON GRADESCOPE > 
    
       

**<span style="color:red">Summary TODO</span>**
- **<span style="color:red">Fill in `CODE BLOCK PCA1`: Fill out `SNIPPET_SIZE`, `PRELENGTH`, `THRESHOLD`, and `EUCLIDEAN_THRESHOLD`</span>**
- **<span style="color:red">Run script in follow cell to get principal components, mean vector, centroids, and test word from the above best case, and then Fill in `CODE BLOCK PCA2` with these values.</span>**
- **<span style="color:red">Fill in `CODE BLOCK PCA3`: Do the actual classification.</span>**

In [ ]:
# Print PCA vectors for Arduino
v = new_basis
for j in range(num_pca):
    a = ("float pca_vec%d[SNIPPET_SIZE]={" % (j+1))
    for i in v[:,j].T:    
        a += str(float(int(i*10000))/10000) + ', '
    a = a[:-2]+"};"
    print(a)
print() 
# Print projetced_mean_vec for Arduino
a = ("float projected_mean_vec[%d]={" % num_pca)    
for j in projected_mean_vec:
    a += str(j) + ', '
a = a[:-2]+"};"
print(a)
print()
# Print the coordinate of all four centroids
for j in range(4):
    a = ("float centroid%d[%d]={" % (j+1, num_pca))
    for i in centroids[j]:
        a += str(i) + ', '
    a = a[:-2]+"};"
    print(a)
print()

for j in range(4):
    print("Test word for word%d(%s) is" % (j+1, all_words_arr[j]))
    a = "int16_t result[SNIPPET_SIZE]={"
    b = np.random.randint(num_samples_test*j, num_samples_test*(j+1)-1)
    for i in processed_A_test[b]:
        a += str(i) + ', '
    a = a[:-2]+"};"
    print(a)

**Try all 4 words as your test word and make sure you are able to get enough accuracy on the Arduino when classifying those words.**

## Questions

<span style="color:#075a04"> **6. Submit a screenshot of your Tinkercad module including codes in `CODE BLOCK PCA3` and classification result from serial monitor to Gradescope.**
    
< YOUR ANSWER ON GRADESCOPE > 
    
       

<a id='part10'></a>
## <span style="color:#ba190f">Part 10: CHECKOFF </span> 
-----

### Submit your answers, files, and screenshots for Questions 1-6 to the Gradescope assignment "Lab 9: SVD/PCA Files and Screenshots." Lab 9 is due on Sunday, August 7 at 11:59pm. No late submissions will be accepted.

### Make sure your circuits are saved properly in Tinkercad. You will need them in the next project phase!

### Remember what each part of your circuit is for by recording this information in a Google doc or somewhere else safe. You will need to write a summary for your final lab report.

### Save this notebook somewhere you can access!

We recommend making a shared Google Drive with your lab partner, since GitHub can be annoying with Jupyter notebooks. It's good to familiarize yourself with the user interface of Tinkercad and understand the basic functions of these circuits that you have built and analyzed.


### Great job finishing 16B Lab 9!